In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [15]:
responses_df = pd.read_csv(r"C:\Users\manig\Downloads\augmentedform.csv")
print(responses_df.shape)
responses_df.head()

(150, 15)


,What do you enjoy doing in your free time?,Consider a project where you have a tight deadline. How do you balance the need to meet that deadline with your commitment to thoroughness and precision?,"Imagine yourself as a game referee. You have made arrived at a decision that you feel is correct. However, one team feels that the decision is incorrect and persistently argues to overturn the decision. How would you proceed in such a situation?","How do you feel about exploring new ideas and concepts, even if they are unconventional?","What workplace do you prefer, and do you prefer working alone or as part of a team and why?","Imagine you are in a situation where you have been offered huge opportunity abroad, however it will be a completely new place and you will not know anyone there. You also have an opportunity in your residing city, which offers less but provides familiarity and comfortable surroundings, which one would you chose and why?",Can you tell how often you feel anxious or nervous in a social situation and how did you handle those situations?,"How do you handle criticism or feedback, especially when it's not delivered in the most constructive way?","Imagine waking up feeling extremely frustrated and irritated for no apparent reason. Early in the morning itself you encounter various challenges and setbacks, such as traffic congestion, a difficult conversation with a colleague, and a technology malfunction. How would your rest of the day go?",How would you describe your work-life balance?,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,"Sleeping, watching stuff, instagram",Prioritise and focus,I'd ask them to approach higher authorities if...,Yeah they should be explored. No room for regr...,"Outdoor workplace, as part of a team",Move abroad,"Very often, deep breaths and calling someone f...",Not take it personally,I'd make sure it goes better,Not aware,75.0,46.0,46.0,69.0,67.0
1,"Listening to music, Reading books",by time management,I would listen to their arguments and only ove...,I mostly find it interesting to explore new id...,I prefer working as a team but I like it bette...,The opportunity in my residing city,I haven't felt much like this lately. In occas...,I do not handle criticism well unless it is pu...,The next few hours of the day might go bad as ...,I would rate it 8/10.,83.0,87.5,27.0,87.5,15.0
2,Listening to music,I make small targets and try to achieve them w...,I would listen to the other side as well and t...,I like exploring new ideas and concepts.,I prefer working in a team because I feel it g...,I will prefer going abroad that is offering me...,I do not often feel anxious in social situatio...,"At first, I feel a bit bad, but then I ponder ...",My rest of the day will go a bit off.,I divide my time between my work and my person...,83.0,75.0,46.0,79.0,56.0
3,Gaming,Use online resources like AI tools to collect ...,Explain the situation to the team and stick to...,Always intrigued in exploring new ideas,Hybrid. Prefer working alone during initial st...,Would choose new place considering the work li...,I do feel anxious. Try to calm myself down and...,Try to understand their point of view. If the ...,"I would try to make sure I stay calm, and not ...",Currently a student so it's quite even,79.0,56.0,67.0,71.0,27.0
4,Watching TV or dancing or learn something new,I always make a list of things I have to do an...,"I will think about it, and ask the Thursday um...",Excited,Team because more ideas and more work can be d...,Abroad,I feel nervous in social situations. To handle...,I ignore it.,Bad,When I am too busy I forget about my family. A...,52.0,77.0,21.0,60.0,92.0


In [19]:
n_questions = 10
answer_df = pd.DataFrame()
score_df = pd.DataFrame()
answer_df['Answers'] = responses_df.iloc[:, :10].values.ravel()
# score_df = pd.concat([] * n_questions, ignore_index=True)
columns_to_repeat = responses_df.columns[10:]
score_df[columns_to_repeat] = np.repeat(responses_df[columns_to_repeat].values, 10, axis=0)
df = pd.concat([answer_df, score_df], axis=1)
df

,Answers,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,"Sleeping, watching stuff, instagram",75.0,46.0,46.0,69.0,67.0
1,Prioritise and focus,75.0,46.0,46.0,69.0,67.0
2,I'd ask them to approach higher authorities if...,75.0,46.0,46.0,69.0,67.0
3,Yeah they should be explored. No room for regr...,75.0,46.0,46.0,69.0,67.0
4,"Outdoor workplace, as part of a team",75.0,46.0,46.0,69.0,67.0
...,...,...,...,...,...,...
1495,opportunity abroad as it gives room for growth.,94.0,83.0,60.0,83.0,25.0
1496,"Occasionally, I deal with them by monitoring m...",94.0,83.0,60.0,83.0,25.0
1497,by revisiting the task.,94.0,83.0,60.0,83.0,25.0
1498,I find it challenging but manage to persevere.,94.0,83.0,60.0,83.0,25.0


In [20]:
X = df['Answers']
# X = df.drop(['Openness ', 'Conscientiousness ', 'Extraversion ', 'Agreeableness ', 'Neuroticism '], axis = 1)
y_O = df['Openness ']
y_C = df['Conscientiousness ']
y_E = df['Extraversion ']
y_A = df['Agreeableness ']
y_N = df['Neuroticism ']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0)

# BERT (Base)

In [33]:
# Dont run this again and again
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_preprocess = hub.KerasLayer(preprocess_url, trainable=True)
bert_encoder = hub.KerasLayer(encoder_url, trainable=True)

In [34]:
rmse = tf.keras.metrics.RootMeanSquaredError()
tf.random.set_seed(42)
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='input_text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
layer = tf.keras.layers.Dropout(0.1, name="dropout_1")(outputs['pooled_output'])
layer = tf.keras.layers.Dense(512, activation='relu', name="dense_1")(outputs['pooled_output'])
layer = tf.keras.layers.Dense(512, activation='relu', name="dense_2")(layer)
layer = tf.keras.layers.Dropout(0.1, name="dropout_2")(layer)
layer = tf.keras.layers.Dense(256, activation='relu', name="dense_3")(layer)
layer = tf.keras.layers.Dense(256, activation='relu', name="dense_4")(layer)
layer = tf.keras.layers.Dropout(0.1, name="dropout_3")(layer)
layer = tf.keras.layers.Dense(128, activation='relu', name="dense_5")(layer)
layer = tf.keras.layers.Dense(128, activation='relu', name="dense_6")(layer)
layer = tf.keras.layers.Dropout(0.1, name="dropout_4")(layer)
layer = tf.keras.layers.Dense(64, activation='relu', name="dense_7")(layer)
layer = tf.keras.layers.Dense(64, activation='relu', name="dense_8")(layer)
layer = tf.keras.layers.Dropout(0.1, name="dropout_5")(layer)
layer = tf.keras.layers.Dense(32, activation='relu', name="dense_9")(layer)
layer = tf.keras.layers.Dense(32, activation='relu', name="dense_10")(layer)
layer = tf.keras.layers.Dropout(0.1, name="dropout_6")(layer)
layer = tf.keras.layers.Dense(16, activation='relu', name="dense_11")(layer)
layer = tf.keras.layers.Dense(16, activation='relu', name="dense_12")(layer)
layer = tf.keras.layers.Dropout(0.1, name="dropou_7")(layer)
layer = tf.keras.layers.Dense(8, activation='relu', name="dense_13")(layer)
layer = tf.keras.layers.Dense(8, activation='relu', name="dense_14")(layer)
layer = tf.keras.layers.Dropout(0.1, name="dropout__8")(layer)
layer = tf.keras.layers.Dense(1, activation='linear', name="output")(layer)
model_O = tf.keras.Model(inputs=[text_input], outputs = [layer])
model_O.compile(optimizer='adam', loss='mean_squared_error', metrics=[rmse,'mae'])
model_O.summary()

ERROR:absl:hub.KerasLayer is trainable but has zero trainable weights.


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_text (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer_7 (KerasLayer)     {'input_word_ids':   0           ['input_text[0][0]']             
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [35]:
Epochs = 20
results_O = model_O.fit(X, y_O, epochs=Epochs)

Epoch 1/20


ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[4096,3072] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node transformer/layer_4/intermediate/einsum/Einsum}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_403774]

In [25]:
model_O.evaluate(X,y_O)

47/47 [==============================] - 14s 271ms/step - loss: 204.0387 - root_mean_squared_error: 14.2842 - mae: 10.6732


[204.0386505126953, 14.284209251403809, 10.673164367675781]

# BERT (Large)

In [26]:
# Dont run this again and again
preprocess_url_large = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url_large = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4"
bert_preprocess_large = hub.KerasLayer(preprocess_url_large)
bert_encoder_large = hub.KerasLayer(encoder_url_large)

In [27]:
rmse = tf.keras.metrics.RootMeanSquaredError()
tf.random.set_seed(42)
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='input_text')
preprocessed_text = bert_preprocess_large(text_input)
outputs = bert_encoder_large(preprocessed_text)
# layer = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
layer = tf.keras.layers.Dense(512, activation='relu', name="dense_1")(outputs['pooled_output'])
layer = tf.keras.layers.Dense(512, activation='relu', name="dense_2")(layer)
layer = tf.keras.layers.Dense(256, activation='relu', name="dense_3")(layer)
layer = tf.keras.layers.Dense(256, activation='relu', name="dense_4")(layer)
layer = tf.keras.layers.Dense(128, activation='relu', name="dense_5")(layer)
layer = tf.keras.layers.Dense(128, activation='relu', name="dense_6")(layer)
layer = tf.keras.layers.Dense(64, activation='relu', name="dense_7")(layer)
layer = tf.keras.layers.Dense(64, activation='relu', name="dense_8")(layer)
layer = tf.keras.layers.Dense(32, activation='relu', name="dense_9")(layer)
layer = tf.keras.layers.Dense(32, activation='relu', name="dense_10")(layer)
layer = tf.keras.layers.Dense(16, activation='relu', name="dense_11")(layer)
layer = tf.keras.layers.Dense(16, activation='relu', name="dense_12")(layer)
layer = tf.keras.layers.Dense(8, activation='relu', name="dense_13")(layer)
layer = tf.keras.layers.Dense(8, activation='relu', name="dense_14")(layer)
layer = tf.keras.layers.Dense(1, activation='linear', name="output")(layer)
large_model_O = tf.keras.Model(inputs=[text_input], outputs = [layer])
large_model_O.compile(optimizer='adam', loss='mean_squared_error', metrics=[rmse,'mae'])
large_model_O.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_text (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_type_ids':   0           ['input_text[0][0]']             
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [28]:
Epochs = 20
large_results_O = large_model_O.fit(X, y_O, epochs=Epochs)

Epoch 1/20
47/47 [==============================] - 50s 751ms/step - loss: 1473.7029 - root_mean_squared_error: 38.3888 - mae: 28.7825
Epoch 2/20
47/47 [==============================] - 37s 778ms/step - loss: 270.9422 - root_mean_squared_error: 16.4603 - mae: 12.0819
Epoch 3/20
47/47 [==============================] - 37s 779ms/step - loss: 250.8141 - root_mean_squared_error: 15.8371 - mae: 11.7434
Epoch 4/20
47/47 [==============================] - 37s 787ms/step - loss: 231.1490 - root_mean_squared_error: 15.2036 - mae: 11.0483
Epoch 5/20
47/47 [==============================] - 37s 792ms/step - loss: 224.2793 - root_mean_squared_error: 14.9760 - mae: 10.9813
Epoch 6/20
47/47 [==============================] - 37s 793ms/step - loss: 230.7589 - root_mean_squared_error: 15.1908 - mae: 11.0497
Epoch 7/20
47/47 [==============================] - 37s 793ms/step - loss: 245.4729 - root_mean_squared_error: 15.6676 - mae: 11.5873
Epoch 8/20
47/47 [==============================] - 37s 797ms